In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time

class PhysicsInformedNN(nn.Module):
    def __init__(self, x0, u0, x1, layers, dt, lb, ub, q):
        super(PhysicsInformedNN, self).__init__()
        
        self.lb = lb
        self.ub = ub
        
        self.x0 = torch.tensor(x0, dtype=torch.float32)
        self.x1 = torch.tensor(x1, dtype=torch.float32)
        
        self.u0 = torch.tensor(u0, dtype=torch.float32)
        
        self.layers = layers
        self.dt = dt
        self.q = max(q,1)
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)
        
        # Load IRK weights
        tmp = np.float32(np.loadtxt('../../Utilities/IRK_weights/Butcher_IRK%d.txt' % (q), ndmin = 2))
        self.IRK_weights = torch.tensor(np.reshape(tmp[0:q**2+q], (q+1,q)), dtype=torch.float32)
        self.IRK_times = torch.tensor(tmp[q**2+q:], dtype=torch.float32)
    
    def initialize_NN(self, layers):
        layers_list = []
        num_layers = len(layers)
        for l in range(num_layers - 1):
            layers_list.append(nn.Linear(layers[l], layers[l + 1]))
            if l != num_layers - 2:
                layers_list.append(nn.Tanh())
        return nn.Sequential(*layers_list)
    
    def forward(self, X):
        return self.neural_net(X)
    
    def neural_net(self, X):
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        return self.weights(H)
    
    def net_U0(self, x):
        U1 = self.neural_net(x)
        U = U1[:, :-1]
        U_x = self.fwd_gradients(U, x)
        U_xx = self.fwd_gradients(U_x, x)
        F = 5.0 * U - 5.0 * U**3 + 0.0001 * U_xx
        U0 = U1 - self.dt * torch.matmul(F, self.IRK_weights.T)
        return U0
    
    def net_U1(self, x):
        U1 = self.neural_net(x)
        U1_x = self.fwd_gradients(U1, x)
        return U1, U1_x
    
    def fwd_gradients(self, U, x):
        g = torch.autograd.grad(U, x, grad_outputs=torch.ones_like(U), create_graph=True)[0]
        return torch.autograd.grad(g, x, grad_outputs=torch.ones_like(g), create_graph=True)[0]
    
    def train(self, nIter):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        for it in range(nIter):
            optimizer.zero_grad()
            U0_pred = self.net_U0(self.x0)
            U1_pred, U1_x_pred = self.net_U1(self.x1)
            loss = torch.sum((self.u0 - U0_pred)**2) + \
                   torch.sum((U1_pred[0, :] - U1_pred[1, :])**2) + \
                   torch.sum((U1_x_pred[0, :] - U1_x_pred[1, :])**2)
            loss.backward()
            optimizer.step()
            
            if it % 10 == 0:
                print('It: %d, Loss: %.3e' % (it, loss.item()))
    
    def predict(self, x_star):
        x_star = torch.tensor(x_star, dtype=torch.float32)
        with torch.no_grad():
            U1_star, _ = self.net_U1(x_star)
        return U1_star.numpy()

# Example usage:
# x0, u0, x1, layers, dt, lb, ub, q = ...  # Define your inputs
# model = PhysicsInformedNN(x0, u0, x1, layers, dt, lb, ub, q)
# model.train(1000)
# x_star = ...  # Define your test data
# predictions = model.predict(x_star)
